In [1]:
from pathlib import Path

import automech as am
import polars as pl
from project_utilities import p_

open_browser = False
tag = "Z_mess_v0"
root_path = Path("../..")

stoich = "C5H7"

calc_mech_json = p_.calculated_pes_mechanism(
    tag, stoich, "json", path=p_.data(root_path)
)
calc_mech = am.io.read(calc_mech_json)

if open_browser:
    disp_mech = calc_mech.model_copy(deep=True)
    disp_mech.reactions = disp_mech.reactions.filter(~pl.col("well_skipping"))
    am.display(disp_mech)

In [2]:
import altair as alt
import numpy as np
from autochem.util import plot
from project_utilities import fig

total = False
legend = False
rate_df1 = fig.rate_data(
    calc_mech,
    reactants=("C5H7(500)",),
    min_branch_frac=0.01,
    colors=[plot.Color.green, plot.Color.purple, plot.Color.red],
)

rate_df2 = fig.rate_data(
    calc_mech,
    reactants=("C5H7(504)",),
    min_branch_frac=0.01,
    colors=[plot.Color.green, plot.Color.blue],
)

_, y_data1 = fig.rate_xy_data(
    rate_df1, T_range=(500, 1200), P=1, total=total, units={"substance": "molecules"}
)
_, y_data2 = fig.rate_xy_data(
    rate_df2, T_range=(500, 1200), P=1, total=total, units={"substance": "molecules"}
)

y_min = np.nanmin([np.nanmin(y_data1), np.nanmin(y_data2)])
y_max = np.nanmax([np.nanmax(y_data1), np.nanmax(y_data2)])
y_range = (y_min, y_max)
print(y_range)

(np.float64(1.757141833622196e-08), np.float64(6767541.073824456))


In [3]:
branch_frac_chart1 = fig.branching_fraction_chart(
    rate_df1,
    P_range=(0.1, 100),
    T=825,
    legend=legend,
    total=True,
)
rate_chart1 = fig.rate_chart(
    rate_df1,
    T_range=(500, 1200),
    P=1,
    legend=legend,
    total=total,
    y_range=y_range,
    units={"substance": "molecules"},
)
chart = alt.vconcat(
    branch_frac_chart1.properties(width=300, height=150),
    rate_chart1.properties(width=300, height=240),
).configure_axis(grid=False)
chart

alt.VConcatChart(...)

In [4]:
branch_frac_chart2 = fig.branching_fraction_chart(
    rate_df2,
    P_range=(0.1, 100),
    T=825,
    legend=legend,
    total=True,
    mark_kwargs={"strokeDash": [8, 10], "strokeCap": "round"},
)
rate_chart2 = fig.rate_chart(
    rate_df2,
    T_range=(500, 1200),
    P=1,
    legend=legend,
    total=total,
    y_range=y_range,
    mark_kwargs={"strokeDash": [8, 10], "strokeCap": "round"},
    units={"substance": "molecules"},
)
chart = alt.vconcat(
    branch_frac_chart2.properties(width=300, height=150),
    rate_chart2.properties(width=300, height=240),
).configure_axis(grid=False)
chart

alt.VConcatChart(...)

In [5]:
branch_frac_chart = alt.layer(branch_frac_chart1, branch_frac_chart2).resolve_scale(
    color="independent"
)
rate_chart = alt.layer(rate_chart1, rate_chart2).resolve_scale(color="independent")

chart = alt.vconcat(
    branch_frac_chart.properties(width=300, height=150),
    rate_chart.properties(width=300, height=240),
).configure_axis(grid=False)
chart

alt.VConcatChart(...)